In [42]:
import pandas as pd
import numpy as np
import re

# NLTK
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# SKLearn
from sklearn.model_selection import train_test_split

In [43]:
df = pd.read_csv('datasets/youtube_video_dataset.csv')

In [44]:
colunas = list()
for coluna in df.columns:
    colunas.append(coluna)
df = df.iloc[:, [0, 2, 3]]
colunas = list()
for coluna in df.columns:
    colunas.append(coluna)
df = df.dropna()

Colunas: Title Videourl Category Description
Número de Linhas: 11211
Colunas: Title Category Description
Número de Linhas com Valores Nulos: 83
Número de Linhas Após Remoção de Nulos: 11128
Número de Linhas com Valores Nulos (Verificação): 0


In [45]:
lista_categorias = df['Category'].value_counts().index.to_list()
dict_mapeamento = dict()
for i in range(len(lista_categorias)):
    dict_mapeamento[lista_categorias[i]] = i
df['Category'] = df['Category'].map(dict_mapeamento)

{'travel blog': 0,
 'Science&Technology': 1,
 'Food': 2,
 'manufacturing': 3,
 'Art&Music': 4,
 'History': 5}

In [46]:
df['Title'] = df['Title'].apply(lambda x: x.lower()) # Tirando letras minúsculas
df['Description'] = df['Description'].apply(lambda x: x.lower())
df['Title'] = df['Title'].apply(lambda x: re.sub("[^a-zA-Z]"," ", x)) # Filtrando símbolos
df['Description'] = df['Description'].apply(lambda x: re.sub("[^a-zA-Z]"," ", x))

In [47]:
#import nltk
#nltk.download('punkt')
#https://datascience.stackexchange.com/questions/25004/text-classifier-with-multiple-bag-of-words
lista_stopwords = set(stopwords.words('english'))
ps = PorterStemmer()

lista_frases = list()

for i, row in df.iterrows():
    lista_palavra = word_tokenize(row['Title'])
    lista_palavra.extend(word_tokenize(row['Description']))
    lista_string = []
    for palavra in lista_palavra:
        if palavra not in lista_stopwords:
            palavra = ps.stem(palavra) # Stemming da palavra
            lista_string.append(palavra)
    string = " ".join(lista_string)
    lista_frases.append(string)

array_frases = np.array(lista_frases)

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(array_frases).astype(np.uint8)
Y = df['Category'].values

In [55]:
from scipy.optimize import linprog
 
dic = {'0': 'travel blog',
 '1': 'Science&Technology',
 '2':'Food',
 '3': 'manufacturing',
 '4': 'Art&Music',
 '5': 'History'}


[2 2 2 ... 4 4 4]
-1


<ipython-input-55-f4cadfe2d985>:11: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  Y = np.where(Y == '0', 1, -1)


In [ ]:
for i in dic.keys():
    df["newTarget"] = np.where(df['Target'] == i, 1 , -1)
     
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    tmp = df.iloc[:,[2,3]].values
    tmp = sc.fit_transform(tmp)
 
    xx = np.array(df.newTarget.values.reshape(-1,1) * tmp)
    t = np.where(df['Target'] == i, 1 , -1)
     
    #2-D array which, when matrix-multiplied by x, gives the values of 
    #the upper-bound inequality constraints at x.
    A_ub = np.append(xx, t.reshape(-1,1), 1)
     
    #1-D array of values representing the upper-bound of each 
    #inequality constraint (row) in A_ub.
    b_ub = np.repeat(-1, A_ub.shape[0]).reshape(-1,1)
     
    # Coefficients of the linear objective function to be minimized.
    c_obj = np.repeat(1, A_ub.shape[1])
    res = linprog(c=c_obj, A_ub=A_ub, b_ub=b_ub,
                  options={"disp": False})
     
    if res.success:
        print('There is linear separability between {} and the rest'.format(dic[i]))
    else:
        print('No linear separability between {} and the rest'.format(dic[i]))